# Парсинг данных с сайта https://familio.org/ с использованием библиотеки BeautifulSoup

In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import xml.etree.ElementTree as ET
import json

In [2]:
final_data = pd.DataFrame()

In [ ]:
def Familio_Parser(url): 
    global final_data
    response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
    page1 = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
    links = page1.findAll('script')
    data = pd.read_json(links[3].text)
    data1 = pd.DataFrame(data=data['props']['pageProps'])
    data2 = pd.DataFrame(data=data1['records']['hydra:member'])
    data2 = data2.drop({
        'uuid',
        'is_comment',
        'owner_level',
        'start_coord_margin_left',
        'start_coord_margin_top',
        'end_coord_margin_left',
        'end_coord_margin_top',
        'parent_id',
        'settlements',
        '@id',
        'page',
        'parent'
        }, axis=1)
    final_data = pd.concat([final_data, data2])
    return final_data

In [ ]:
urls = pd.read_csv('URLS.csv')

In [ ]:
# def Familio_Parser_gubernia(url): 
response = requests.get('https://familio.org/knowledge-base/catalogs/nmyg1901?catalogUUID=nmyg1901&page-records=1') # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page1 = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
links = page1.findAll('script')
data = pd.read_json(links[3].text)
data1 = pd.DataFrame(data=data['props']['pageProps'])
data2 = pd.DataFrame(data=data1['records']['hydra:view'])

In [12]:
data1

,initialReduxState,initPropsId,catalogInfo,currentGroupInfo,childGroups,searchRecords,records
auth,{'profile': {}},0.316642,NaN,None,NaN,None,NaN
@context,NaN,0.316642,/contexts/Catalog,None,NaN,None,NaN
@id,NaN,0.316642,/catalogs/6c0c374c-ea61-4837-97f7-a6963b5056b5,None,/catalogs/nmyg1901/level1,None,/catalogs/nmyg1901/records
@type,NaN,0.316642,Catalog,None,NaN,None,NaN
name,NaN,0.316642,Список населённых мест Ярославской губернии 19...,None,NaN,None,NaN
hidden,NaN,0.316642,False,None,NaN,None,NaN
key,NaN,0.316642,nmyg1901,None,NaN,None,NaN
description,NaN,0.316642,Список населенных мест Ярославской губернии / ...,None,NaN,None,NaN
issueYearDescription,NaN,0.316642,1901,None,NaN,None,NaN
issueYearOrder,NaN,0.316642,1901,None,NaN,None,NaN


In [ ]:
links[3].text

Code

In [132]:
# словарь уездов задаем вручную
uezd_dict = {
    '528ff402-3d0c-4d21-8db2-c384f6a1271b':"Ярославский уезд",
    '564dea6b-63a5-4491-b6fa-e75af3977d61':"Даниловский уезд",
    '66cce446-6f61-41c0-975d-c13064bbff78':"Любимский уезд",
    '765c8a72-1aea-4302-876b-1c74d79266ae':"Мологский уезд",
    '9b82a766-f04f-4af5-9797-887d7d828717':"Мышкинский уезд",
    'ba884fae-e86c-47a0-a662-4d23735b1101':"Пошехонский уезд",
    'c3b224e1-5297-4d87-bff0-5b793ec1821b':"Романово-Борисоглебский уезд",
    'c4291bc2-af32-4884-ae30-9eabbdb7d444':"Ростовский уезд",
    'd99c1a74-aabc-46bc-88d8-5317faad0a3d':"Рыбинский уезд",
    'f259747d-7a35-4b5a-a03f-61066f09ff6e':"Угличский уезд"
}

In [134]:
def Familio_Guber_Parser(url):
    # gubernia page parsing function
    # return uezd list for 1 page
    response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
    page1 = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
    links = page1.findAll('script')
    data = pd.read_json(links[3].text)
    data1 = pd.DataFrame(data=data['props']['pageProps'])
    data2 = pd.DataFrame(data=data1['childGroups']['hydra:member'])
    if data2.shape[0] > 0:
        vol_list = data2[['uuid','name_ru']]
    else:
        vol_list = pd.DataFrame()
    return vol_list

In [145]:
vol_df = pd.DataFrame() # пустой фрейм для списка уездов
#
url_part_1 = 'https://familio.org/knowledge-base/catalogs/nmyg1901?catalogUUID=nmyg1901&page-records=1' # URL страницы Яр губ
uezd_uuid_list = list(uezd_dict.keys())
uezd_name_list = list(uezd_dict.values())
#
for u in range(len(uezd_uuid_list)):
    # цикл по списку уездов
    for i in range(3): # цикл по страницам со списком волостей уезда
        url_part_2 = '&page-group=' + str(i+1) + '&level1='
        url = url_part_1 + url_part_2 + uezd_uuid_list[u]
        vol_df_ = Familio_Guber_Parser(url)
        vol_df_['uezd_name'] = uezd_name_list[u]
        vol_df_['uezd_uuid'] = uezd_uuid_list[u]
        vol_df_['uezd_url'] = url
        if not vol_df_.empty:
            vol_df = pd.concat([vol_df, vol_df_], axis=0, ignore_index=True)

# в итоге получаем датафрейм со списком волостей по уезду

In [ ]:
vol_df.head()

# 2 level

In [ ]:
# посчитаем сколько страниц для каждой вол
# https://familio.org/knowledge-base/catalogs/nmyg1901?page-records=1&page-group=1&level1=528ff402-3d0c-4d21-8db2-c384f6a1271b&level2=00902776-d347-4996-af91-8a45177d3d54
vol_count = vol_df.shape[0] # volost quantity
vol_df['pages'] = 0 # создаем столбец под количество страниц в волости
for v in range(vol_count):
    # cycle in volost list
    url = 'https://familio.org/knowledge-base/catalogs/nmyg1901?page-records=1&page-group=1&level1=' + vol_df['uezd_uuid'].loc[v] + '&level2=' + vol_df['uuid'].loc[v]
    response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
    page1 = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
    links = page1.findAll('script')
    data = pd.read_json(links[3].text)
    data1 = pd.DataFrame(data=data['props']['pageProps'])
    a = data1['currentGroupInfo']['settlement_count']
    if a % 10 == 0:
        b = a // 10
    else:
        b = a // 10 + 1
    vol_df['pages'].loc[v] = b # заполняем столбец значениями

In [181]:
final_data = pd.DataFrame() # подготовим пустой датафрейм для итогового списка

In [188]:
def Familio_Uezd_Parser(url):
    # функция парсит страницу и заполняет датафрейм
    global final_data
    response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
    page1 = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
    links = page1.findAll('script')
    data = pd.read_json(links[3].text)
    data1 = pd.DataFrame(data=data['props']['pageProps'])
    if not data1.empty:
        data2 = pd.DataFrame(data=data1['records']['hydra:member'])
        if not data2.empty:
            data2 = data2.drop({
            'start_coord_margin_left',
            'start_coord_margin_top',
            'end_coord_margin_left',
            'end_coord_margin_top',
            '@id',
            'page',
            'parent'
            }, axis=1)
            final_data = pd.concat([final_data, data2])

In [189]:
vol_count = vol_df.shape[0] # volost quantity
url_part_1 = 'https://familio.org/knowledge-base/catalogs/nmyg1901?level1='
for v in range(vol_count):
    # cycle in volost list
    for i in range(vol_df['pages'].loc[v]):
        # cycle in pages of volost
        url = url_part_1 + vol_df['uezd_uuid'].loc[v] + '&page-records=' + str(i+1) + '&page-group=1&level2='+ vol_df['uuid'].loc[v] + '&catalogUUID=nmyg1901'
        Familio_Uezd_Parser(url)

# в итоге получаем DF со списком np

In [190]:
final_data.to_csv('output_data.csv') # выгрузка в csv файл

try

In [ ]:
# prepare urls list for each page of villages
urls_list = []
vol_count = vol_df.shape[0] # volost quantity
url_part_1 = 'https://familio.org/knowledge-base/catalogs/nmyg1901?level1='
for v in range(vol_count):
    # cycle in volost list
    url = url_part_1 + vol_df['uezd_uuid'].loc[v] + '&page-records=1&page-group=1&level2='+ vol_df['uuid'].loc[v] + '&catalogUUID=nmyg1901'
    urls_list.append(url)